In [2]:
import warnings
warnings.filterwarnings('ignore')

from datasets import Dataset
import pandas as pd
# import numpy as np

# from tqdm import tqdm
import torch

from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [18]:
language = 'gujarati'

In [3]:
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer
MODEL_CHECKPOINT ="google/mt5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_CHECKPOINT, )
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
encodes = tokenizer.encode('મારું આખું શરીર થીજી ગયું છે.')
outputs = tokenizer.decode(encodes)
outputs
# encodes

'મારું આખું શરીર થીજી ગયું છે.</s>'

In [6]:
df = pd.read_csv('post_asr_datas/pseudo_train_gujarati.csv')
df.dropna(inplace=True)
df.head()

,file,og_transcription,ms_asr_transcription
0,../data/audios/gujarati/train/guj_audio_0.wav,જ્યારે હું તેણીને યાદ કરું છું ત્યારે હું નિરા...,જયારે હું તેણીને યાદ કરું છું ત્યારે હું નિરાશ...
1,../data/audios/gujarati/train/guj_audio_1.wav,જ્યારે હું ભારે વસ્તુઓ વહન કરું છું ત્યારે મને...,જ્યારે હું ભારે વસ્તુઓબાન કરું છું ત્યારે મને ...
2,../data/audios/gujarati/train/guj_audio_2.wav,જ્યારે હું મારા હાથને ખસેડું છું ત્યારે ખૂબ પી...,જ્યારે હું મારા હાથને ખસેડું છું ત્યારે ખૂબ પી...
3,../data/audios/gujarati/train/guj_audio_3.wav,મારા પુત્રએ તેના હોઠને વીંધી નાખ્યું હતું અને ...,મારા પુત્રે તેના હોઠને વીંધી નાખ્યું હતું અને ...
4,../data/audios/gujarati/train/guj_audio_4.wav,મારી પીઠના નીચેના ભાગમાં સ્નાયુઓમાં દુખાવો થાય છે,મારી પીઠના નીચેના ભાગમાં સ્નાયુઓમાં દુખાવો થાય.


In [7]:
SOURCE = 'ms_asr_transcription'

TARGET = 'og_transcription'

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df[SOURCE], df[TARGET], test_size=0.10,
                                                    shuffle=True,
                                                    random_state=100)

In [9]:
def prep_data_for_model_fine_tuning(source_lang: list, target_lang: list) -> list:
    data_dict = dict()
    data_dict['data'] = []

    for sr_text, tr_text in zip(source_lang, target_lang):
        temp_dict = dict()
        temp_dict[SOURCE] = sr_text
        temp_dict[TARGET] = tr_text

        data_dict['data'].append(temp_dict)

    return data_dict

In [10]:
training_data = prep_data_for_model_fine_tuning(x_train.values, y_train.values)
testing_data = prep_data_for_model_fine_tuning(x_test.values, y_test.values)

In [11]:
training_data['data'][:5]

[{'ms_asr_transcription': 'હું મારા માથા પર ખૂબ જ દબાણ અનુભવું છું.',
  'og_transcription': 'હું મારા માથા પર ખૂબ જ દબાણ અનુભવું છું.'},
 {'ms_asr_transcription': 'ચાલ્યા પછી મને મારા પગ નીચે થોડો દુખાવો થાય છે.',
  'og_transcription': 'ચાલ્યા પછી મને મારા પગ નીચે થોડો દુખાવો થાય છે'},
 {'ms_asr_transcription': 'જ્યારે હું હિલ્સ પહેરું છું ત્યારે મારા પગની કમાન દુખે છે.',
  'og_transcription': 'જ્યારે હું હીલ્સ પહેરું છું ત્યારે મારા પગની કમાનો દુખે છે'},
 {'ms_asr_transcription': 'જયારે હું સવારે જાગું છું ત્યારે મને વિચિત્ર લાગે છે અને ચક્કર આવે છે.',
  'og_transcription': 'જ્યારે હું સવારે જાગું છું ત્યારે મને વિચિત્ર લાગે છે અને ચક્કર આવે છે'},
 {'ms_asr_transcription': 'આપને ઝડપથી મટાડવામાં મદદ કરેં છે.',
  'og_transcription': 'કાપને ઝડપથી મટાડવામાં શું મદદ કરે છે?'}]

In [13]:
MAX_INPUT_LENGTH = 128

def generate_model_ready_dataset(dataset: list, source: str, target: str, tokenizer):
    preped_data = []

    for row in dataset:
        inputs = row[source]
        targets = row[target]

        model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)#, padding=True)
        model_inputs['data'] = row

        # setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=MAX_INPUT_LENGTH,
                                 truncation=True, padding=True)
            model_inputs['labels'] = labels['input_ids']

        preped_data.append(model_inputs)

    return preped_data

In [14]:
train_data = generate_model_ready_dataset(dataset=training_data['data'],
                                        tokenizer=tokenizer,
                                        source=SOURCE,
                                        target=TARGET)

test_data = generate_model_ready_dataset(dataset=testing_data['data'],
                                        tokenizer=tokenizer,
                                        source=SOURCE,
                                        target=TARGET)

In [15]:
train_df = pd.DataFrame.from_records(train_data)
test_df = pd.DataFrame.from_records(test_data)

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   attention_mask  414 non-null    object
 1   data            414 non-null    object
 2   input_ids       414 non-null    object
 3   labels          414 non-null    object
dtypes: object(4)
memory usage: 13.1+ KB


In [16]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset

Dataset({
    features: ['attention_mask', 'data', 'input_ids', 'labels'],
    num_rows: 414
})

In [19]:
MODEL_INTENT = 'asr_correction'

original_model = MODEL_CHECKPOINT.split('/')[-1]
trained_model_name = f'{original_model}-finetuned-{MODEL_INTENT}-{language}'
batch_size = 8
model_args = Seq2SeqTrainingArguments(
    trained_model_name,
    # evaluation_strategy='epoch',
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.02,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()

cuda:0


In [21]:
trainer = Seq2SeqTrainer(
    model.to(device),
    model_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

  0%|          | 0/52 [00:00<?, ?it/s]

{'train_runtime': 674.3538, 'train_samples_per_second': 0.614, 'train_steps_per_second': 0.077, 'train_loss': 7.273348294771635, 'epoch': 1.0}


TrainOutput(global_step=52, training_loss=7.273348294771635, metrics={'train_runtime': 674.3538, 'train_samples_per_second': 0.614, 'train_steps_per_second': 0.077, 'total_flos': 44847135590400.0, 'train_loss': 7.273348294771635, 'epoch': 1.0})

In [22]:
trainer.save_model(f"{MODEL_CHECKPOINT}-finetuned-{MODEL_INTENT}-{language}")